# Quantization Examples

Loading [AdamLucek/Orpo-Llama-3.2-1B-15k](https://huggingface.co/AdamLucek/Orpo-Llama-3.2-1B-15k) regularly, then quantized to [8-Bit](https://arxiv.org/abs/2208.07339), and quantized to [4-Bit](https://arxiv.org/abs/2305.14314)

In [ ]:
%%capture
!pip install -U transformers bitsandbytes accelerate

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

---
# Quantizing and Converting to GGUF Format

[GGUF](https://huggingface.co/docs/hub/gguf) is a specific format developed by Georgi Gerganov in [GGML](https://github.com/ggerganov/ggml?tab=readme-ov-file), specifically for the [llama.cpp](https://github.com/ggerganov/llama.cpp) library. It has become a popular way of storing and distributing language models for a variety of downstream tasks, and comes with built in quantization capabilities.

## Quantization methods

The names of the quantization methods follow the naming convention: "q" + the number of bits + the variant used. K represents the [k-quants quantization method](https://github.com/ggerganov/llama.cpp/pull/1684), a specific quantization method developed for LLMs that handles different parts of the model differently based on their sensitivity to quantization. Here's a simplified writeup of popular methods

* `q2_k`: Uses Q4_K for the attention.vw and feed_forward.w2 tensors, Q2_K for the other tensors.
* `q3_k_l`: Uses Q5_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K
* `q3_k_m`: Uses Q4_K for the attention.wv, attention.wo, and feed_forward.w2 tensors, else Q3_K
* `q3_k_s`: Uses Q3_K for all tensors
* `q4_0`: Original quant method, 4-bit.
* `q4_1`: Higher accuracy than q4_0 but not as high as q5_0. However has quicker inference than q5 models.
* `q4_k_m`: Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K
* `q4_k_s`: Uses Q4_K for all tensors
* `q5_0`: Higher accuracy, higher resource usage and slower inference.
* `q5_1`: Even higher accuracy, resource usage and slower inference.
* `q5_k_m`: Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K
* `q5_k_s`:  Uses Q5_K for all tensors
* `q6_k`: Uses Q8_K for all tensors
* `q8_0`: Almost indistinguishable from float16. High resource use and slow. Not recommended for most users.

We'll be using q4_k_m for 4-bit quantization using the k-quants method, using the outline [here](https://github.com/ggerganov/llama.cpp/blob/master/examples/quantize/README.md)



In [ ]:
# Install llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
!pip install -r llama.cpp/requirements.txt

fatal: destination path 'llama.cpp' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
  Using cached transformers-4.57.6-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.36.2-py3-none-any.whl.metadata (15 kB)
Using cached transformers-4.57.6-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.36.2-py3-none-any.whl (566 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 1.4.1
    Uninstalling huggingface_hub-1.4.1:
      Successfully uninstalled huggingface_hub-1.4.1
  Attempting unin

In [ ]:
%cd llama.cpp
!make

/content/llama.cpp
Makefile:6: *** Build system changed:
 The Makefile build has been replaced by CMake.

 For build instructions see:
 https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md

.  Stop.


In [ ]:
# Download model
%cd .
!curl -sSfL https://hf.co/git-xet/install.sh | sh


/content/llama.cpp
Detected OS: Linux
Detected Architecture: x86_64
Unzipping...
Setting executable permissions...
Installing to /usr/local/bin...
git-xet installed to global config!
Installation complete!
Cleaning up...


In [ ]:

!rm -rf Orpo-Llama-3.2-1B-15k

In [ ]:
!git clone https://huggingface.co/AdamLucek/Orpo-Llama-3.2-1B-15k

Cloning into 'Orpo-Llama-3.2-1B-15k'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45 (from 1)
Unpacking objects: 100% (45/45), 20.02 KiB | 2.50 MiB/s, done.


In [ ]:
# convert the model to ggml FP16 format
!python convert_hf_to_gguf.py Orpo-Llama-3.2-1B-15k

INFO:hf-to-gguf:Loading model: Orpo-Llama-3.2-1B-15k
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:hf-to-gguf:heuristics detected float16 tensor dtype, setting --outtype f16
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {32}
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {2048, 128258}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float16 --> F16, shape = {8192, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {2048, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {2048, 8192}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,       

In [ ]:
# quantize the model to 4-bits (using Q4_K_M method)
!llama.cpp/llama-quantize Orpo-Llama-3.2-1B-15k/Llama-3.2-1B-15k-F16.gguf ./Orpo-Llama-3.2-1B-15k/Orpo-Llama-3.2-1B-15k-Q4_K_M.gguf Q4_K_M

/bin/bash: line 1: llama.cpp/llama-quantize: No such file or directory


In [ ]:
!cmake -B build
!cmake --build build --config Release


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1") 
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_

In [ ]:

# quantize the model to 4-bits (using Q4_K_M method)
!./build/bin/llama-quantize Orpo-Llama-3.2-1B-15k/Llama-3.2-1B-15k-F16.gguf ./Orpo-Llama-3.2-1B-15k/Orpo-Llama-3.2-1B-15k-Q4_K_M.gguf Q4_K_M


main: build = 8037 (b48e80f67)
main: built with GNU 11.4.0 for Linux x86_64
main: quantizing 'Orpo-Llama-3.2-1B-15k/Llama-3.2-1B-15k-F16.gguf' to './Orpo-Llama-3.2-1B-15k/Orpo-Llama-3.2-1B-15k-Q4_K_M.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 44 key-value pairs and 147 tensors from Orpo-Llama-3.2-1B-15k/Llama-3.2-1B-15k-F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                     general.sampling.top_p f32              = 0.900000
llama_model_loader: - kv   3:                      general.sampling.temp f32              = 0.600000
llama_model_loader: - kv   4:                               general.name str              = Llama 3.2 1B
llama_model_loader: - kv  

In [ ]:

# List contents of the model directory to find the generated GGUF file
!ls Orpo-Llama-3.2-1B-15k


config.json			   README.md
generation_config.json		   special_tokens_map.json
Llama-3.2-1B-15k-F16.gguf	   tokenizer_config.json
model.safetensors		   tokenizer.json
Orpo-Llama-3.2-1B-15k-Q4_K_M.gguf


## Command Line Testing

`llama.cpp/llama-cli -m Orpo-Llama-3.2-1B-15k/Orpo-Llama-3.2-1B-15k-Q4_K_M.gguf -cnv -p "You are a helpful assistant"`

Model has been uploaded to [AdamLucek/Orpo-Llama-3.2-1B-15k-Q4_K_M-GGUF](https://huggingface.co/AdamLucek/Orpo-Llama-3.2-1B-15k-Q4_K_M-GGUF)

Alternatively, you can use the [gguf-my-repo](https://huggingface.co/spaces/ggml-org/gguf-my-repo) space to quickly convert your HF model to GGUF format with any quantization method.

In [ ]:

!./build/bin/llama-cli -m Orpo-Llama-3.2-1B-15k/Orpo-Llama-3.2-1B-15k-Q4_K_M.gguf -cnv -p "which is the commonly used language in world?"



Loading model... |-\|/-\|/-\|/-\|/-\|/-\|/- 


▄▄ ▄▄
██ ██
██ ██  ▀▀█▄ ███▄███▄  ▀▀█▄    ▄████ ████▄ ████▄
██ ██ ▄█▀██ ██ ██ ██ ▄█▀██    ██    ██ ██ ██ ██
██ ██ ▀█▄██ ██ ██ ██ ▀█▄██ ██ ▀████ ████▀ ████▀
                                    ██    ██
                                    ▀▀    ▀▀

build      : b8037-b48e80f67
model      : Orpo-Llama-3.2-1B-15k-Q4_K_M.gguf
modalities : text

available commands:
  /exit or Ctrl+C     stop or exit
  /regen              regenerate the last response
  /clear              clear the chat history
  /read               add a text file


> which is the commonly used language in world?

|- According to the World Economic Forum's Global Information Technology Report 2022, the most commonly used language in the world is English. The United States has the largest share of English speakers globally, with approximately 400 million people who speak it as their first language. However, there are many other languages that are us

In [31]:

!./build/bin/llama-cli -m Llama-3.2-1B-15k-F16.gguf -cnv -p "which is the commonly used language in world?"


AGENTS.md		       grammars
AUTHORS			       include
benches			       LICENSE
build			       licenses
build-xcframework.sh	       Makefile
ci			       media
CLAUDE.md		       models
cmake			       mypy.ini
CMakeLists.txt		       Orpo-Llama-3.2-1B-15k
CMakePresets.json	       pocs
CODEOWNERS		       poetry.lock
common			       pyproject.toml
CONTRIBUTING.md		       pyrightconfig.json
convert_hf_to_gguf.py	       README.md
convert_hf_to_gguf_update.py   requirements
convert_llama_ggml_to_gguf.py  requirements.txt
convert_lora_to_gguf.py        scripts
docs			       SECURITY.md
examples		       src
flake.lock		       tests
flake.nix		       tools
ggml			       vendor
gguf-py

Loading model... |gguf_init_from_file: failed to open GGUF file 'Llama-3.2-1B-15k-F16.gguf' (No such file or directory)
llama_model_load: error loading model: llama_model_loader: failed to load model from Llama-3.2-1B-15k-F16.gguf
llama_model_load_from_file_impl: failed to load model
llama_params_fit: encountered an e